# Create station pairs with rebalance counts
Data will be used to draw arcs via pydeck + deck.gl

In [ ]:
import pandas as pd
import pydeck as pdk
import dash_deck
import os

# import our helpers module. appending parent directory ("..") to sys.path might be necessary for helpers to be imported on some systems
import sys

sys.path.insert(0, "..")
import helpers

In [ ]:
DATA_DIR = "data/"
STATIONS_DIR = DATA_DIR + "stations/"
REBALANCE_DIR = DATA_DIR + "rebalance_parquet/"
STATIONS_PAIRS_DIR = DATA_DIR + "stations-pairs/"

TRIP_YEAR = 2019

In [ ]:
# read stations
stations = pd.read_csv(STATIONS_DIR + "stations.csv", index_col=0)
stations["stationid"] = stations["stationid"].astype("int64")
stations.drop(
    ["capacity", "neighbourhood", "boro", "zipcode", "elevation_ft"],
    axis=1,
    inplace=True,
)
stations

,stationid,stationname,latitude,longitude
0,455,1 Ave & E 44 St,40.750020,-73.969053
1,434,9 Ave & W 18 St,40.743174,-74.003664
2,491,E 24 St & Park Ave S,40.740964,-73.986022
3,384,Fulton St & Waverly Ave,40.683178,-73.965964
4,474,5 Ave & E 29 St,40.745168,-73.986831
...,...,...,...,...
1425,3685,Prospect Park - 5 Year Anniversary Celebration,40.660652,-73.964590
1426,3695,E 5 St & 2 Ave,40.726870,-73.989190
1427,3700,E 87 St & 3 Ave,40.779406,-73.953336
1428,3805,E 80 St & Park Ave,40.776173,-73.959757


In [ ]:
rebpairs = pd.read_parquet(
    REBALANCE_DIR + "rebalance_pairs" + helpers.PARQUET_EXTENSION, engine="pyarrow"
)
rebpairs = rebpairs.loc[rebpairs.rebal_year == TRIP_YEAR].drop("rebal_year", axis=1)
rebpairs["stationid_from"] = rebpairs["stationid_from"].astype("int")
rebpairs["stationid_to"] = rebpairs["stationid_to"].astype("int")
rebpairs = rebpairs[["stationid_from", "stationid_to", "rebal_count"]]
rebpairs = rebpairs.sort_values(by="rebal_count", ascending=False).head(100)
rebpairs

,stationid_from,stationid_to,rebal_count
42,445,394,1807
57,3263,432,1531
68,445,433,1307
71,3443,3158,1276
73,445,3718,1226
...,...,...,...
726,335,432,292
735,426,3177,289
741,3718,317,286
745,265,473,285


In [ ]:
_from = pd.merge(
    left=rebpairs,
    right=stations,
    left_on="stationid_from",
    right_on="stationid",
    how="left",
)
_from.drop("stationid", axis=1, inplace=True)
_from.rename(
    columns={
        "latitude": "latitude_from",
        "longitude": "longitude_from",
        "stationname": "stationname_from",
    },
    inplace=True,
)
_from

,stationid_from,stationid_to,rebal_count,stationname_from,latitude_from,longitude_from
0,445,394,1807,E 10 St & Avenue A,40.727408,-73.981420
1,3263,432,1531,Cooper Square & E 7 St,40.729236,-73.990868
2,445,433,1307,E 10 St & Avenue A,40.727408,-73.981420
3,3443,3158,1276,W 52 St & 6 Ave,40.761330,-73.979820
4,445,3718,1226,E 10 St & Avenue A,40.727408,-73.981420
...,...,...,...,...,...,...
95,335,432,292,Washington Pl & Broadway,40.729039,-73.994046
96,426,3177,289,West St & Chambers St,40.717548,-74.013221
97,3718,317,286,E 11 St & Avenue B,40.727464,-73.979504
98,265,473,285,Stanton St & Chrystie St,40.722293,-73.991475


In [ ]:
_to = pd.merge(
    left=_from, right=stations, left_on="stationid_to", right_on="stationid", how="left"
)
_to.rename(
    columns={
        "latitude": "latitude_to",
        "longitude": "longitude_to",
        "stationname": "stationname_to",
    },
    inplace=True,
)
_to.insert(0, "rebal_count", _to.pop("rebal_count"))
_to

,rebal_count,stationid_from,stationid_to,stationname_from,latitude_from,longitude_from,stationid,stationname_to,latitude_to,longitude_to
0,1807,445,394,E 10 St & Avenue A,40.727408,-73.981420,394,E 9 St & Avenue C,40.725213,-73.977688
1,1531,3263,432,Cooper Square & E 7 St,40.729236,-73.990868,432,E 7 St & Avenue A,40.726218,-73.983799
2,1307,445,433,E 10 St & Avenue A,40.727408,-73.981420,433,E 13 St & Avenue A,40.729554,-73.980572
3,1276,3443,3158,W 52 St & 6 Ave,40.761330,-73.979820,3158,W 63 St & Broadway,40.771639,-73.982614
4,1226,445,3718,E 10 St & Avenue A,40.727408,-73.981420,3718,E 11 St & Avenue B,40.727464,-73.979504
...,...,...,...,...,...,...,...,...,...,...
95,292,335,432,Washington Pl & Broadway,40.729039,-73.994046,432,E 7 St & Avenue A,40.726218,-73.983799
96,289,426,3177,West St & Chambers St,40.717548,-74.013221,3177,W 84 St & Broadway,40.786795,-73.977112
97,286,3718,317,E 11 St & Avenue B,40.727464,-73.979504,317,E 6 St & Avenue B,40.724537,-73.981854
98,285,265,473,Stanton St & Chrystie St,40.722293,-73.991475,473,Rivington St & Chrystie St,40.721101,-73.991925


In [ ]:
station_pairs = _to
station_pairs

,rebal_count,stationid_from,stationid_to,stationname_from,latitude_from,longitude_from,stationid,stationname_to,latitude_to,longitude_to
0,1807,445,394,E 10 St & Avenue A,40.727408,-73.981420,394,E 9 St & Avenue C,40.725213,-73.977688
1,1531,3263,432,Cooper Square & E 7 St,40.729236,-73.990868,432,E 7 St & Avenue A,40.726218,-73.983799
2,1307,445,433,E 10 St & Avenue A,40.727408,-73.981420,433,E 13 St & Avenue A,40.729554,-73.980572
3,1276,3443,3158,W 52 St & 6 Ave,40.761330,-73.979820,3158,W 63 St & Broadway,40.771639,-73.982614
4,1226,445,3718,E 10 St & Avenue A,40.727408,-73.981420,3718,E 11 St & Avenue B,40.727464,-73.979504
...,...,...,...,...,...,...,...,...,...,...
95,292,335,432,Washington Pl & Broadway,40.729039,-73.994046,432,E 7 St & Avenue A,40.726218,-73.983799
96,289,426,3177,West St & Chambers St,40.717548,-74.013221,3177,W 84 St & Broadway,40.786795,-73.977112
97,286,3718,317,E 11 St & Avenue B,40.727464,-73.979504,317,E 6 St & Avenue B,40.724537,-73.981854
98,285,265,473,Stanton St & Chrystie St,40.722293,-73.991475,473,Rivington St & Chrystie St,40.721101,-73.991925


In [ ]:
if not os.path.exists(STATIONS_PAIRS_DIR):
    os.mkdir(STATIONS_PAIRS_DIR)

station_pairs.to_csv(STATIONS_PAIRS_DIR + "pairs" + helpers.CSV_EXTENSION)

In [ ]:
GREEN_RGB = [0, 255, 0, 40]
RED_RGB = [240, 100, 0, 40]
TOOLTIP_TEXT = {
    "html": "{rebal_count} bikes to be rebalanced <br/>from {station_name_from} to {station_name_to}"
}
token = open(".mapbox_token").read()

In [ ]:
# Specify a deck.gl ArcLayer
arc_layer = pdk.Layer(
    "ArcLayer",
    data=station_pairs,
    get_width="S000 * 10",
    # get_width="rebal_count",
    get_source_position=["longitude_from", "latitude_from"],
    get_target_position=["longitude_to", "latitude_to"],
    get_tilt=10,
    get_source_color=RED_RGB,
    get_target_color=GREEN_RGB,
    pickable=True,
    auto_highlight=True,
)

In [ ]:
view_state = pdk.ViewState(
    latitude=station_pairs.latitude_to.mean(),
    longitude=station_pairs.longitude_to.mean(),
    bearing=0,
    pitch=30,
    zoom=12,
)  # TODO fix
r = pdk.Deck(
    arc_layer, initial_view_state=view_state, tooltip=TOOLTIP_TEXT, map_style="dark"
)
r.to_html("arc_layer.html")

# rv = dash_deck.DeckGL(r.to_json(),style = {'height' : '100%',"position": 'relative'},
#                       id='rebalancing-strategy-graphic',
#                       mapboxKey=mapbox_key)

In [ ]:
import dash
from dash import html

app = dash.Dash(__name__)

app.layout = html.Div(
    dash_deck.DeckGL(r.to_json(), id="deck-gl", tooltip=TOOLTIP_TEXT, mapboxKey=token)
)

app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
